In [5]:
import numpy as np
from scipy.optimize import minimize

# ----------  basic 2×2 SU(2) block in Euler‑angle form ----------
def su2(alpha, beta, gamma):
    c = np.cos(beta / 2)
    s = np.sin(beta / 2)
    return np.array(
        [[np.exp(-0.5j * (alpha + gamma)) * c, -np.exp(-0.5j * (alpha - gamma)) * s],
         [np.exp( 0.5j * (alpha - gamma)) * s,  np.exp( 0.5j * (alpha + gamma)) * c]],
        dtype=complex,
    )

# ----------  embed that 2×2 block on the |0>↔|i> spoke ----------
def embed(mat, i, d):
    E = np.eye(d, dtype=complex)
    E[np.ix_([0, i], [0, i])] = mat        # replace 2×2 sub‑block
    return E

# ----------  minimal‑length star‑rotation solver ----------
def star_solver(U):
    d = U.shape[0]
    m = (d * d - 1 + 2) // 3               #  ⌈(d²−1)/3⌉
    spokes = [(k % (d - 1)) + 1 for k in range(m)]   # 1,2,…,d−1,1,2,…

    def build(x):                          # x is a flat length‑3m vector
        V = np.eye(d, dtype=complex)
        for k, i in enumerate(spokes):
            V = embed(su2(*x[3 * k : 3 * k + 3]), i, d) @ V
        return V

    def loss(x):
        diff = build(x) - U
        return np.real(np.vdot(diff, diff))          # ‖diff‖_F²

    x0 = 0.01 * np.random.randn(3 * m)               # small random start
    res = minimize(loss, x0, method="L-BFGS-B", options={"maxiter": 10000000})
    if not res.success:
        raise RuntimeError("optimiser failed: " + res.message)
    return res.x.reshape(m, 3)                        # (θ,φ,χ) triples

# ----------  example: 8‑level Walsh–Hadamard ----------
H2 = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=complex)
H8 = np.kron(np.kron(H2, H2), H2)                    # 8×8 Hadamard, det = 1

params = star_solver(H8)                             # shape (21, 3)
print("gate count:", len(params))                    # 21  =  ⌈(8²−1)/3⌉


gate count: 21


In [6]:
print(params)

[[-0.81451616  1.54465733 -3.155273  ]
 [-1.3500931   0.4292489  -1.11118347]
 [ 1.28727346 -0.3274037   0.07541974]
 [ 0.18292368 -1.15529809  2.52967091]
 [-0.20420662 -0.92293739  1.13484341]
 [ 0.94365847 -4.74230736  0.69451812]
 [ 0.72561273  0.72782261 -3.26232662]
 [-0.27707429  0.46012246 -0.06893178]
 [-0.83147627 -1.95278282 -0.585057  ]
 [-0.67736139 -0.11417929  1.31956801]
 [ 0.54639838  0.85562126  0.3760464 ]
 [-1.37689395 -1.36454966  0.76768468]
 [ 1.18813823 -1.23680246  0.7257697 ]
 [-1.35157469  1.63116011 -1.26169918]
 [ 0.24493202  1.80860069 -2.21807724]
 [-0.97349327  0.53076714 -1.41523709]
 [-1.95547475 -1.96263173 -0.04483913]
 [ 2.50392577 -1.27746197  0.13981865]
 [-0.78025146 -2.06363295  0.48016036]
 [ 0.54906461 -2.3598904   2.14974621]
 [-0.00566075  1.56323566 -1.14005133]]


In [8]:
import numpy as np
from scipy.optimize import minimize

def su2(a,b,c):
    t=np.cos(b/2); s=np.sin(b/2)
    return np.array([[np.exp(-0.5j*(a+c))*t,-np.exp(-0.5j*(a-c))*s],
                     [np.exp(0.5j*(a-c))*s, np.exp(0.5j*(a+c))*t]],dtype=complex)

def embed(m,i,d):
    E=np.eye(d,dtype=complex)
    E[np.ix_([0,i],[0,i])]=m
    return E

def star_solver(U,maxiter=1000):
    d=U.shape[0]
    m=(d*d-1+2)//3
    idx=[(k%(d-1))+1 for k in range(m)]
    def build(x):
        V=np.eye(d,dtype=complex)
        for k,i in enumerate(idx):
            V=embed(su2(*x[3*k:3*k+3]),i,d)@V
        return V
    loss=lambda x:np.linalg.norm(build(x)-U)**2
    x0=0.01*np.random.randn(3*m)
    res=minimize(loss,x0,method='L-BFGS-B',options={'maxiter':10000000})
    return res.x.reshape(m,3),idx

def params_to_matrices(params,idx,d):
    return [embed(su2(*p),i,d) for p,i in zip(params,idx)]

def reconstruct(params,idx,d):
    V=np.eye(d,dtype=complex)
    for M in params_to_matrices(params,idx,d):
        V=M@V
    return V

# example: 8‑level Hadamard
H2=(1/np.sqrt(2))*np.array([[1,1],[1,-1]],dtype=complex)
H8=np.kron(np.kron(H2,H2),H2)

params,idx=star_solver(H8)
mats=params_to_matrices(params,idx,8)
U_rec=reconstruct(params,idx,8)
err=np.linalg.norm(U_rec-H8)
print(len(mats),'matrices  |  error =',err)


21 matrices  |  error = 1.4853621179011605
